In [2]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
import pickle as pkl
from input_output import read_csv, write_csv
from utils import get_df_info, get_current_time
from bs4 import BeautifulSoup
import codecs
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.pipeline import Pipeline
from functools import reduce
from collections import Counter
from wordcloud import WordCloud
from lda import LDA
import re

logging.getLogger('matplotlib').setLevel(logging.ERROR)

logger = logging.getLogger(__name__)

if not os.path.exists('./logs'):
    os.mkdir('./logs')

In [3]:
df = pd.read_csv('token.csv')

In [4]:
df

,OBJECT_ID,title_token,text_token
0,379746418,"['公募', '基金', '二季', '报重', '仓股', '全', '揭露', '，',...","['香港', '万得', '通讯社', '报道', '，', '随着', '月日', '基金..."
1,453849759,"['民航局', '将', '开展', '无人机', '适航', '审定', '分级', '管理']","['据', '民航局', '网站日', '消息', '，', '为', '促进', '国',..."
2,453856916,"['民航局', '推动', '形成', '基于', '运行', '风险', '的', '民用...","['公司', '讯', '，', '月日', '民航局', '印发', '基于', '运行'..."
3,453941181,"['促进', '民营', '经济', '发展', '地', '设', '纾困', '基金',...","['多', '省市', '出台', '措施', '意见', '促进', '民营', '经济'..."
4,453948030,"['国资委', '鼓励', '外资', '参与', '国企', '改革', '，', '业内...","['证券', '日报', '报道', '，', '近日', '，', '国资委会', '同家..."
5,453949829,"['趣墅', '时隔', '一', '年', '再', '获', '千万', '融资', '...","['环球', '旅讯', '成立', '于', '年月', '的', '趣墅', '，', ..."
6,453954720,"['融资客', '青睐股', '，', '豫能', '控股', '买入', '占', '比'...","['香港', '万得', '通讯社', '报道', '，', '月日', '，', '数据'..."
7,453955303,"['年', '哈萨克斯坦', '铁合金', '产量', '同比', '增长']","['阿斯塔纳', '月日', '消息', '，', '哈萨克斯坦', '统计', '委员会'..."
8,453956430,"['中银', '国际', '每日', '速递', '顺鑫', '农业', '；', '基金'...","['目录', '宏观', '策略', '策略', '基金', '四季度', '持仓', '报..."
9,453957310,"['蓝色', '起源', '开始', '建厂', '生产', '火箭', '发动机']","['品玩', '月日', '报道', '，', '蓝色', '起源', '已', '正式',..."


In [5]:
from zhon.hanzi import punctuation
remove_set = set(punctuation).union(set(["?", ":", ":", "万得", "通讯社", "报道"]))
lines = codecs.open('data/chinese_stopwords.txt', 'r', 'utf-8').readlines()
stopwords = set()
for line in lines:
    words = line.replace('\n','').split(',')
    words = [re.sub(r'\s+', '', w) for w in words if w!='']
    stopwords = stopwords.union(set(words))
kill_set = remove_set.union(stopwords)

In [6]:
df['text_token_r'] = df.text_token.apply(lambda x: [w for w in eval(x) if w not in kill_set and len(w)>1])

In [7]:
df

,OBJECT_ID,title_token,text_token,text_token_r
0,379746418,"['公募', '基金', '二季', '报重', '仓股', '全', '揭露', '，',...","['香港', '万得', '通讯社', '报道', '，', '随着', '月日', '基金...","[香港, 月日, 基金, 季报, 披露, 最后, 节点, 过去, 目前, 基金, 重仓股, ..."
1,453849759,"['民航局', '将', '开展', '无人机', '适航', '审定', '分级', '管理']","['据', '民航局', '网站日', '消息', '，', '为', '促进', '国',...","[民航局, 网站日, 消息, 促进, 民用, 无人机, 产业, 健康, 成长, 月日, 民航..."
2,453856916,"['民航局', '推动', '形成', '基于', '运行', '风险', '的', '民用...","['公司', '讯', '，', '月日', '民航局', '印发', '基于', '运行'...","[公司, 月日, 民航局, 印发, 运行, 风险, 无人机, 适航, 审定, 指导, 意见,..."
3,453941181,"['促进', '民营', '经济', '发展', '地', '设', '纾困', '基金',...","['多', '省市', '出台', '措施', '意见', '促进', '民营', '经济'...","[省市, 出台, 措施, 意见, 促进, 民营, 经济, 发展, 明确, 暂时, 困难, 企..."
4,453948030,"['国资委', '鼓励', '外资', '参与', '国企', '改革', '，', '业内...","['证券', '日报', '报道', '，', '近日', '，', '国资委会', '同家...","[证券, 日报, 近日, 国资委会, 同家, 中央, 企业, 瑞士, 达沃斯, 出席, 世界..."
5,453949829,"['趣墅', '时隔', '一', '年', '再', '获', '千万', '融资', '...","['环球', '旅讯', '成立', '于', '年月', '的', '趣墅', '，', ...","[环球, 旅讯, 成立, 年月, 趣墅, 地产商, 别墅, 业主, 合作, 运营, 别墅, ..."
6,453954720,"['融资客', '青睐股', '，', '豫能', '控股', '买入', '占', '比'...","['香港', '万得', '通讯社', '报道', '，', '月日', '，', '数据'...","[香港, 月日, 数据, 统计, 显示, 收盘, 共有, 只个, 融资, 资金, 买入, 融..."
7,453955303,"['年', '哈萨克斯坦', '铁合金', '产量', '同比', '增长']","['阿斯塔纳', '月日', '消息', '，', '哈萨克斯坦', '统计', '委员会'...","[阿斯塔纳, 月日, 消息, 哈萨克斯坦, 统计, 委员会, 周三, 公布, 数据, 显示,..."
8,453956430,"['中银', '国际', '每日', '速递', '顺鑫', '农业', '；', '基金'...","['目录', '宏观', '策略', '策略', '基金', '四季度', '持仓', '报...","[目录, 宏观, 策略, 策略, 基金, 四季度, 持仓, 报告, 重仓, 股集, 中度, ..."
9,453957310,"['蓝色', '起源', '开始', '建厂', '生产', '火箭', '发动机']","['品玩', '月日', '报道', '，', '蓝色', '起源', '已', '正式',...","[品玩, 月日, 蓝色, 起源, 正式, 美国, 阿拉巴马州, 亨茨维尔, 建厂, 新格伦号..."


In [8]:
def filtered_word_cloud(df, truncate=50):
    c = Counter(reduce(lambda x, y: x+y, df.text_token_r))
    text_freq_s = sorted(c.items(), key=lambda x: x[1], reverse=True)
    arr = np.array(text_freq_s)
    df_t = pd.DataFrame(arr, columns={'word','occurance'})
    df_t.word = df_t.word.astype('int')
    return df_t
df_t = filtered_word_cloud(df)

In [10]:
words = set(df_t[(df_t.word > 0) & (df_t.word < 1800)].occurance.values)
#words = set(df_t.occurance.values)

In [11]:
df['text_token_r'] = df.text_token_r.apply(lambda x: ' '.join([w for w in x if w in words]))
df

,OBJECT_ID,title_token,text_token,text_token_r
0,379746418,"['公募', '基金', '二季', '报重', '仓股', '全', '揭露', '，',...","['香港', '万得', '通讯社', '报道', '，', '随着', '月日', '基金...",香港 月日 基金 季报 披露 最后 节点 过去 目前 基金 重仓股 已经 显露 涉及 上市 ...
1,453849759,"['民航局', '将', '开展', '无人机', '适航', '审定', '分级', '管理']","['据', '民航局', '网站日', '消息', '，', '为', '促进', '国',...",民航局 网站日 消息 促进 民用 无人机 产业 健康 成长 月日 民航局 印发 运行 风险 ...
2,453856916,"['民航局', '推动', '形成', '基于', '运行', '风险', '的', '民用...","['公司', '讯', '，', '月日', '民航局', '印发', '基于', '运行'...",月日 民航局 印发 运行 风险 无人机 适航 审定 指导 意见 无人机 运行 场景 丰富 运...
3,453941181,"['促进', '民营', '经济', '发展', '地', '设', '纾困', '基金',...","['多', '省市', '出台', '措施', '意见', '促进', '民营', '经济'...",省市 出台 措施 意见 促进 民营 经济 发展 明确 暂时 困难 稳贷 续贷 纾困 基金 规...
4,453948030,"['国资委', '鼓励', '外资', '参与', '国企', '改革', '，', '业内...","['证券', '日报', '报道', '，', '近日', '，', '国资委会', '同家...",日报 近日 国资委会 同家 中央 瑞士 达沃斯 出席 世界 经济 论坛 年年会 活动 论坛 ...
5,453949829,"['趣墅', '时隔', '一', '年', '再', '获', '千万', '融资', '...","['环球', '旅讯', '成立', '于', '年月', '的', '趣墅', '，', ...",环球 旅讯 成立 年月 趣墅 地产商 别墅 业主 合作 运营 别墅 度假村 整合 热门 旅游...
6,453954720,"['融资客', '青睐股', '，', '豫能', '控股', '买入', '占', '比'...","['香港', '万得', '通讯社', '报道', '，', '月日', '，', '数据'...",香港 月日 数据 统计 显示 收盘 共有 只个 融资 资金 买入 融资 买入额 当日 成交 ...
7,453955303,"['年', '哈萨克斯坦', '铁合金', '产量', '同比', '增长']","['阿斯塔纳', '月日', '消息', '，', '哈萨克斯坦', '统计', '委员会'...",阿斯塔纳 月日 消息 哈萨克斯坦 统计 委员会 周三 公布 数据 显示 年月 哈萨克斯坦 铁...
8,453956430,"['中银', '国际', '每日', '速递', '顺鑫', '农业', '；', '基金'...","['目录', '宏观', '策略', '策略', '基金', '四季度', '持仓', '报...",目录 宏观 策略 策略 基金 四季度 持仓 报告 重仓 股集 中度 下降 是否 趋势 海外月...
9,453957310,"['蓝色', '起源', '开始', '建厂', '生产', '火箭', '发动机']","['品玩', '月日', '报道', '，', '蓝色', '起源', '已', '正式',...",品玩 月日 蓝色 起源 正式 美国 阿拉巴马州 亨茨维尔 建厂 新格伦号 联合 发射 联盟 ...


In [13]:
obj_df = pd.read_csv('code_obj.csv').rename(columns={'obj':'OBJECT_ID'})

In [18]:
merge_df = obj_df.merge(df, on='OBJECT_ID', how='inner').sort_values('code')

In [ ]:
T

In [20]:
def test_sum(series):
    return reduce(lambda x, y: x + y, series)
merge_df.groupby('code', as_index=False).agg({'text_token_r': test_sum})

,code,text_token_r
0,000001.SZ,香港 月日 数据 统计 显示 收盘 共有 只个 融资 资金 买入 融资 买入额 当日 成交 ...
1,000002.SZ,房企 共有 产权房 地块 限竞 地块 出现 泾渭 分明 拿地 态度 观点 产网 临近 春节 ...
2,000004.SZ,本报 记者 陈炜 仓储 运输 迈入 房地产 自生物 医药 转向 互联网 游戏 往前 追溯 深...
3,000009.SZ,原标 回暖 明显 市股 最高 涨幅 白马股 创出 阶段 高附 名单 近期 新三板 出现 回暖...
4,000018.SZ,上市 连续 业绩 预告 爆雷 局面 沪深 交易所 紧急 关注 询函 面对 多家 业绩 爆雷 ...
5,000048.SZ,观点 地产 网讯 月日 深圳市 康达尔 集团 股份 有限公司 公告 宣布 股票 交易 异常 ...
6,000058.SZ,月日 晚间 深赛格 公告 控股 孙公司 深圳市 赛格 城市 建设 发展 有限公司 以下 简称...
7,000069.SZ,深圳 龙岗区 聚焦 数字 硬件 创意 设计 动漫 游戏 影视 传媒 数字 印刷 数字 展示 ...
8,000088.SZ,香港 月日 数据 统计 显示 收盘 共有 只个 融资 资金 买入 融资 买入额 当日 成交 ...
9,000100.SZ,特别 提示 订阅号 涉及 研究 信息 光大 电子 研究 团队 编写 面向 光大 客户 专业 ...


In [21]:
vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, ngram_range=(1,3), max_df=0.3, min_df=10)
tfidf = vectorizer.fit_transform(merge_df.text_token_r)

In [22]:
tfidf.toarray() # filter

array([[0.        , 0.        , 0.        , ..., 0.04724285, 0.05457669,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00732883, 0.00822992, 0.00822992, ..., 0.        , 0.        ,
        0.        ]])

In [23]:
vectorizer.get_feature_names()

['一下',
 '一下 特斯拉',
 '一下 特斯拉 发家史',
 '一二',
 '一二 城市',
 '一二 城市 票房',
 '一以贯之',
 '一以贯之 重资产',
 '一以贯之 重资产 模式',
 '一体',
 '一体 特斯拉',
 '一体 特斯拉 超级',
 '一体化',
 '一半',
 '一季度',
 '一季度 实际',
 '一季度 实际 业绩',
 '一季度 财报',
 '一季度 财报 落地',
 '一定',
 '一定 回收',
 '一定 回收 风险',
 '一定 差距',
 '一定 差距 立案',
 '一定 影响',
 '一定 影响 利亚德',
 '一定 污染',
 '一定 污染 难以',
 '一定 滞后性',
 '一定 滞后性 应用',
 '一定 程度',
 '一定 程度 挤压',
 '一度',
 '一度 濒临',
 '一度 濒临 破产',
 '一次性',
 '一次性 解决',
 '一次性 解决 有利于',
 '一次性 计提',
 '一次性 计提 亏损',
 '一次性 计提 超过',
 '一步步',
 '一步步 成为',
 '一步步 成为 全球',
 '一汽',
 '一汽 富维',
 '一汽 富维 车灯',
 '一波',
 '一波 业绩',
 '一波 业绩 变脸',
 '一流',
 '一流 研发',
 '一流 研发 中心',
 '一直',
 '一直 存在',
 '一直 存在 产能',
 '一直 松下',
 '一直 松下 独家',
 '一直 饱受',
 '一直 饱受 质疑',
 '一级',
 '一级 交易商',
 '一级 交易商 使用',
 '一级 供应商',
 '一级 供应商 销量',
 '一线',
 '一线 城市',
 '一线 城市 人均',
 '一致',
 '一致 行动',
 '一致 行动 国安',
 '一致 行动人',
 '一致 预期约',
 '一致 预期约 高于',
 '一致性',
 '一览',
 '一览 之后',
 '一览 之后 有家',
 '一览 代码',
 '一览 代码 简称',
 '一起',
 '一起 星标',
 '一起 星标 国金',
 '一起 看看',
 '一起 看看 华灿',
 '一起 看看 厦门',
 '一起 看看 珈伟',
 '一起 看看 继续',
 '一起 看看 联建',
 '一路',
 '七交五直',
 '七交五直 核准

In [24]:
plsa = NMF(n_components=10, init='nndsvd')
plsa_matrix = plsa.fit_transform(tfidf)

In [25]:
lsa = TruncatedSVD(n_components=10, algorithm='randomized')
lsa_matrix = lsa.fit_transform(tfidf)

In [26]:
def print_topics(model, vectorizer, top_words=5):
    terms = vectorizer.get_feature_names()
    for i, comp in enumerate(model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:top_words]
        print("Topic "+ str(i) + ": ")
        print(sorted_terms)
        ts = ' '.join([t[0] for t in sorted_terms])
        print(ts)

In [ ]:
#手动过滤： 基金

In [27]:
print_topics(plsa, vectorizer, top_words=5)

Topic 0: 
[('买入额', 0.6281656284041004), ('融资 买入额', 0.6281656284041004), ('买入', 0.5093271558730265), ('当日', 0.3140559959923734), ('共有', 0.313400301476057)]
买入额 融资 买入额 买入 当日 共有
Topic 1: 
[('暂停', 0.6979518746436968), ('暂停 上市', 0.6338926971482959), ('商誉', 0.5260693559867421), ('上市 风险', 0.504310283248788), ('暂停 上市 风险', 0.37015004466652)]
暂停 暂停 上市 商誉 上市 风险 暂停 上市 风险
Topic 2: 
[('高压', 0.7263778478515187), ('设备', 0.3112327040911706), ('增速', 0.28965369923433004), ('直流', 0.25184753461424375), ('领域', 0.2485656085585949)]
高压 设备 增速 直流 领域
Topic 3: 
[('商誉', 0.37799831730778544), ('计提', 0.35229550236785856), ('准备', 0.25804725598993133), ('报告期', 0.24318158340022786), ('评估', 0.20318548753631738)]
商誉 计提 准备 报告期 评估
Topic 4: 
[('计算机', 0.5928201218381784), ('基金', 0.5926588802327227), ('持仓', 0.45446742661102174), ('计算机 板块', 0.2613868265806043), ('基金 持股', 0.2561658161034092)]
计算机 基金 持仓 计算机 板块 基金 持股
Topic 5: 
[('折价率', 0.3038983219024414), ('光威', 0.22266273044227833), ('光威 复材', 0.22266273044227833), ('复材', 0.2226

In [28]:
print_topics(lsa, vectorizer, top_words=5)

Topic 0: 
[('买入额', 0.251897494642772), ('融资 买入额', 0.251897494642772), ('买入', 0.20708371641449122), ('共有', 0.1292329121046123), ('当日', 0.12914492384618229)]
买入额 融资 买入额 买入 共有 当日
Topic 1: 
[('商誉', 0.24623700281935929), ('计提', 0.17330283288574125), ('暂停', 0.15393324843357534), ('商誉 减值', 0.138401310099295), ('暂停 上市', 0.13785530628996379)]
商誉 计提 暂停 商誉 减值 暂停 上市
Topic 2: 
[('暂停', 0.18621312387019687), ('暂停 上市', 0.17195097371326168), ('上市 风险', 0.1371053860762331), ('暂停 上市 风险', 0.10072332293923617), ('保壳', 0.07554249220442719)]
暂停 暂停 上市 上市 风险 暂停 上市 风险 保壳
Topic 3: 
[('计提', 0.1285038608586847), ('报告期', 0.11241844142183557), ('准备', 0.10012668289417054), ('评估', 0.09472750118263934), ('商誉', 0.09163243584386585)]
计提 报告期 准备 评估 商誉
Topic 4: 
[('计算机', 0.2768078351686275), ('基金', 0.2765441683356751), ('持仓', 0.21220069219236334), ('计算机 板块', 0.12268471791102308), ('基金 持股', 0.1201358919042348)]
计算机 基金 持仓 计算机 板块 基金 持股
Topic 5: 
[('折价率', 0.15055540414070884), ('席位', 0.11326855509522094), ('光威', 0.11067511304516

In [29]:
cv = CountVectorizer(ngram_range=(1,3), max_df=0.3, min_df=10)
X = cv.fit_transform(df.text_token_r)

In [30]:
lda = LDA(n_topics=10, random_state=0)
lda_matrix = lda.fit_transform(X)

INFO:lda:n_documents: 980
INFO:lda:vocab_size: 4359
INFO:lda:n_words: 307208
INFO:lda:n_topics: 10
INFO:lda:n_iter: 2000
/opt/anaconda3/lib/python3.7/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -3236935
INFO:lda:<10> log likelihood: -2440299
INFO:lda:<20> log likelihood: -2344382
INFO:lda:<30> log likelihood: -2315244
INFO:lda:<40> log likelihood: -2299198
INFO:lda:<50> log likelihood: -2288216
INFO:lda:<60> log likelihood: -2279806
INFO:lda:<70> log likelihood: -2273850
INFO:lda:<80> log likelihood: -2268438
INFO:lda:<90> log likelihood: -2264690
INFO:lda:<100> log likelihood: -2263034
INFO:lda:<110> log likelihood: -2261493
INFO:lda:<120> log likelihood: -2260945
INFO:lda:<130> log likelihood: -2258960
INFO:lda:<140> log likelihood: -2258

INFO:lda:<1920> log likelihood: -2250691
INFO:lda:<1930> log likelihood: -2250257
INFO:lda:<1940> log likelihood: -2250157
INFO:lda:<1950> log likelihood: -2251039
INFO:lda:<1960> log likelihood: -2250702
INFO:lda:<1970> log likelihood: -2250654
INFO:lda:<1980> log likelihood: -2251508
INFO:lda:<1990> log likelihood: -2251213
INFO:lda:<1999> log likelihood: -2251975


In [31]:
print_topics(lda, cv, top_words=5)

Topic 0: 
[('期货', 0.02304887989786006), ('上市', 0.01603275912949636), ('机构', 0.015386537479778651), ('科创板', 0.012617016123845613), ('管理', 0.011324572824410194)]
期货 上市 机构 科创板 管理
Topic 1: 
[('业绩', 0.0260099523666765), ('股份', 0.018216781181634712), ('上市', 0.01716957380364472), ('净利润', 0.016706854264532865), ('股东', 0.011787414953975237)]
业绩 股份 上市 净利润 股东
Topic 2: 
[('基金', 0.0491700762996033), ('银行', 0.0355164196564806), ('保险', 0.030401454630053965), ('融资', 0.025038991295897015), ('资金', 0.019099031910369314)]
基金 银行 保险 融资 资金
Topic 3: 
[('营业部', 0.032050162192597584), ('金额', 0.024634925263209487), ('换手率', 0.022875377517252993), ('买入', 0.020361737880172284), ('成交', 0.02023605589831825)]
营业部 金额 换手率 买入 成交
Topic 4: 
[('价格', 0.014330320468484124), ('下降', 0.011870114303207474), ('指数', 0.011692573652105034), ('增速', 0.010728781546120367), ('预期', 0.009283093387143367)]
价格 下降 指数 增速 预期
Topic 5: 
[('利率', 0.09427298216373559), ('实际', 0.05771716975984319), ('总额', 0.05682556457926044), ('方式', 0.055696198017188

In [32]:
plsa.components_

array([[3.79134117e-05, 0.00000000e+00, 0.00000000e+00, ...,
        1.07155372e-01, 6.83672593e-02, 5.74762610e-02],
       [2.67475441e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.24249374e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.71140272e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.05400614e-02, 1.13694395e-02, 1.13694395e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
<0.1,0.1,,>
<>